In [1]:
from dataLoader import PointCloudLoader
import torch
from net import RadarNet
from trainer import train

In [2]:
# 总类别数
num_classes = 20
# 学习率
lr = 1e-4
# 训练轮数
epoch = 50
# 每个点云数据从中取num_len个区域
num_len = 1
# 每个区域的大小
area_size = (16 * 10, 16 * 10)
# 最高高程长度num_z个
num_z = 100
# 平面和高程分辨
xy_resolution = 0.5
z_resolution = 0.5
# 编码长度
embed_size = 32
# 隐变量长度
num_hiddens = 32
# 循环神经网络层数
num_layers = 2
# 读取数据线程数量
num_workers = 1
# 训练阶段每一次输入的序列数量(分块计算会减少显存使用，但是会减速，仅在训练阶段有意义)
num_seq = 16 * 16 * 2 * 2
# num_seq = None

train_dir = "./data_chunk/train"
test_dir = "./data_chunk/test"
device = "cuda"

In [3]:
# 训练集要打乱，每一块生成num_len个随机区域，一共是总文件数*num_len个
train_dataloader = PointCloudLoader(
    root_path=train_dir,
    num_classes=num_classes,
    num_len=num_len,
    area_size=area_size,
    num_z=num_z,
    xy_resolution=xy_resolution,
    z_resolution=z_resolution,
    random=True,
)
# 测试集不打乱，按顺序来一块一块分类
test_dataloader = PointCloudLoader(
    root_path=test_dir,
    num_classes=num_classes,
    num_len=num_len,
    area_size=area_size,
    num_z=num_z,
    xy_resolution=xy_resolution,
    z_resolution=z_resolution,
    random=False,
)
test_loader = torch.utils.data.DataLoader(
    test_dataloader, batch_size=1, num_workers=num_workers
)
train_loader = torch.utils.data.DataLoader(train_dataloader, batch_size=1)

net = RadarNet(
    num_classes=num_classes,
    elevation_resolution=num_z,
    embed_size=embed_size,
    num_hiddens=num_hiddens,
    num_layers=num_layers,
    num_seq=num_seq,
    features=[64, 128, 256, 512, 1024],
    dropout=0.2,
)

weight = torch.load("best_model_train.pth")
net.load_state_dict(weight, strict=False)
# weight = torch.load("best_model_32.pth")
# net.load_state_dict(weight, strict=False)

optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)

In [4]:
# from utils import eval_net_point
# eval_net_point(net, test_loader, device)

In [5]:
from utils import pred_file

result, header = pred_file(
    net,
    "data/train/data/WMSC_points - Cloud.las",
    area_size,
    num_z,
    xy_resolution,
    z_resolution,
    resolution=1000,
    device=device,
    only_pred=True,
)

In [6]:
from utils import save_result
save_result(
    "result.las",
    header,
    torch.tensor(result),
)